# Bertelsmann hackaweekendthon


## I. Introduction

We chose to look at Vermont Uniform Hospital Discharge Data Sets (http://www.healthvermont.gov/health-statistics-vital-records/health-care-systems-reporting/hospital-discharge-data). They provide emergency, expanded outpatients, inpatients, outpatients discharge information for 2006-2016.

Vermont Blueprint for Health Women's Health Initiative (http://blueprintforhealth.vermont.gov/about-blueprint/womens-health-initiative): <br>
"Women receive substantial preventive care services in OB-GYN and women’s health clinic settings. Through the Women’s Health Initiative, women’s health specialty providers are providing enhanced health and psychosocial screening along with comprehensive family planning counseling and timely access to long acting reversible contraception (LARC). New staff, training, and payments support effective follow-up to provider screenings through brief, in-office intervention and referral to services for mental health, substance use disorder, trauma, partner violence, food and housing. Since launch, the Women's Health Initiative has expanded to include Blueprint Patient Centered Medical Homes.

A few key supports can help Women's Health providers and Patient Centered Medical Homes be even more effective in providing preventive care, identifying health and social risks, connecting women to community supports, and helping ensure more pregnancies are intentional.

Currently in Vermont, **half of all pregnancies are unintended**. Unintended pregnancies are associated with increased risk of poor health outcomes for mothers and babies and long-term negative consequences for the health and wellbeing of the children and adults those babies become.

The Healthy Vermonters 2020 goal for pregnancy intention is 65%."

One of their initiative is providing same-day access to birth control. On August 2012, Affordable Care Act of 2010 mandated that contraceptives be provided free. 

The question is: **Did providing free access to birth control via Affordable Care Act correlate with increased usage of contraceptives and fewer pregnancy complications?**

The hypothesis is that it does. Therefore, **if the hypothesis is true, there would be high number of discharge records related to contraceptives and less on pregnancy complicatoins after 2012**.


## II. Data wrangling

### A. Examine data
Let's take a look at 2016 outpatient data. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

outpatient16 = pd.read_csv('VTOUTP16.TXT')
outpatient16.info()

/Users/rosaria/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,5,54,55,56,57) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370633 entries, 0 to 370632
Data columns (total 70 columns):
hnum2         370633 non-null int64
ATYPE         370633 non-null int64
asour         370633 non-null object
intage        370633 non-null int64
TXTZIP        370633 non-null object
sex           370633 non-null object
dstat         370633 non-null int64
PPAY          370633 non-null int64
CHRGS         370633 non-null float64
DX1           370633 non-null object
DX2           370633 non-null object
DX3           370633 non-null object
DX4           370633 non-null object
DX5           370633 non-null object
DX6           370633 non-null object
DX7           370633 non-null object
DX8           370633 non-null object
DX9           370633 non-null object
DX10          370633 non-null object
DX11          370633 non-null object
DX12          370633 non-null object
DX13          370633 non-null object
DX14          370633 non-null object
DX15          370633 non-null object
DX16 

From this I want: 
- sex (need to be int)
- ccsdx (need to be int): diagnosis classification
- DY: year
- DISCD_QTR: quarter 

In [2]:
print(outpatient16['sex'].unique())

[2 1 '1' '2' ' ']


In [3]:
print(outpatient16['ccsdx'].unique())

[200 47 258 208 138 155 86 153 124 92 257 251 225 139 205 204 90 126 154
 143 24 59 211 127 99 239 102 244 229 137 201 232 136 55 122 149 259 7 230
 172 253 84 235 236 133 108 54 233 128 250 93 657 171 240 83 659 91 159
 241 125 163 651 176 118 106 181 167 94 160 101 100 120 197 58 22 4 96 131
 231 142 245 670 152 146 98 144 653 255 252 661 135 147 212 237 23 256 134
 175 217 162 161 44 32 82 50 170 238 164 29 184 95 49 48 36 87 169 185 46
 11 166 173 198 215 121 103 213 203 12 10 650 15 28 14 191 116 183 158 80
 25 51 207 88 247 38 210 129 81 114 56 57 8 89 41 119 652 140 254 663 79
 33 209 19 39 141 16 115 26 27 40 132 157 42 62 9 113 168 195 199 196 228
 37 117 111 174 17 234 662 97 130 246 660 182 145 224 2 186 248 165 177
 242 3 21 105 202 243 109 151 60 187 2617 18 206 112 85 222 52 43 178 123
 107 148 53 658 '205' '250' '168' '239' '163' '151' '7' '136' '660' '197'
 '165' '159' '126' '245' '251' '235' '255' '94' '650' '211' '83' '232'
 '106' '54' '238' '95' '162' '90' '155' '244

Sex and ccsdx are a mixture of int, str (with ' '). For sex, will look for 2 & '2'. For ccsdx, will take out ' ' and force int datatype.

### B. Data import

Write a script that will allow importing of all other datasets. All formats are fairly consistent throughout according to PUF_FILE_LAYOUT_and_CODES. 2006 does not have qtr information.

Get only the columns I need (to work with smaller data), and filter for female with ccsdx between 176 and 196.

interesting ccsdx:
- 176: contraceptive and procreative management 
- 177: spontaneous abortion 
- 178: induced abortion 
- 179: postabortion complications 
- 183: hypertension complicating pregnancy 
- 184: early or threatened labor 
- 186: gestational diabetes 
- 190: fetal distress and abnormal forces of labor 
- 196: normal pregnancy and/or delivery

ccsdx that should happen more due to genetic/chance than due to policy change
- 180: ectopic pregnancy
- 185: prolonged pregnancy 
- 187: malposition, malpresentation
- 192: umbilical cord complication
- 193: OB related trauma to perineum and vulva

In [4]:
# import inpatient data
interesting_dx = [176,177,178,179,183,184,186,190,196]
uninteresting_dx = [180, 185, 187, 192, 193]

def read_patient(fileName):
    '''reads VT patient file'''
    df = pd.read_csv(fileName)
    df.rename(columns={'sex': 'SEX', 'ccsdx': 'CCSDX', 'dy': 'DY'}, inplace=True)
    df = df['SEX,CCSDX,DY,DISCD_QTR'.split(',')]
    # only female
    df = df[df['SEX'].map(lambda x: x == 2 or x == '2')]
    # filter out empty ccsdx & fix type
    df = df[df['CCSDX'].map(lambda x: x != ' ' and x != '  ')]
    df['CCSDX'] = df['CCSDX'].astype('int64')
    # only ccsdx I want as defined above
    df = df[df['CCSDX'].map(lambda x: x in interesting_dx or x in uninteresting_dx)]
    # change column names
    return df

In [5]:
# 2010 files have missing headers
def add_header(fileName2011, fileName2010):
    with open(fileName2011, 'r') as f:
        header = f.readline()
    newFileName = fileName2010.split('.')[0]+'_head.txt'
    output = open(newFileName, 'w')
    output.write(header)
    with open(fileName2010, 'r') as f:
        line = f.readline()
        while line != '':
            output.write(line)
            line = f.readline()
    output.close()

add_header('VTINP11.TXT', 'VTINP10.TXT')

In [6]:
in09 = read_patient('VTINP09.txt')
in10 = read_patient('VTINP10_head.txt')
in11 = read_patient('VTINP11.TXT')
in12 = read_patient('VTINP12.TXT')
in13 = read_patient('VTINP13.TXT')
in14 = read_patient('VTINP14.TXT')
in15 = read_patient('VTINP15_revised.TXT')
in16 = read_patient('VTINP16.TXT')

inpatient = pd.concat([in09,in10,in11,in12,in13,in14,in15,in16], ignore_index=True)
inpatient.to_csv('2009-2016inpatient.csv')

/Users/rosaria/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (8,64) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/rosaria/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (9,10,11,58,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/rosaria/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (2,8,10,11,32,58,59,60,61,62,63,64,76) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/rosaria/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (2,10,11,58,60) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/rosaria/anaconda3/lib/py

In [7]:
ex09 = read_patient('2009VTEXPOUTP_PUF.TXT')
add_header('VTEXPOUTP11.TXT', 'VTEXPOUTP10.TXT')
ex10 = read_patient('VTEXPOUTP10_head.txt')
ex11 = read_patient('VTEXPOUTP11.TXT')
ex12 = read_patient('VTEXPOUTP12.TXT')
ex13 = read_patient('VTEXPOUTP13.TXT')
ex14 = read_patient('VTEXPOUTP14.TXT')
ex15 = read_patient('VTEXPOUTP15_revised.TXT')
ex16 = read_patient('VTEXPOUTP16.TXT')

extended = pd.concat([ex09,ex10,ex11,ex12,ex13,ex14,ex15,ex16], ignore_index=True)
extended.to_csv('2009-2016extendedPatient.csv')

/Users/rosaria/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (8,51,52,53,69) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/rosaria/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (2,9,29,54,55,56,57) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/rosaria/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (2,8,54,55,71) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/rosaria/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (2,5,8,54,55,71) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/rosaria/anaconda3/lib/python3.6/site

In [8]:
out09 = read_patient('VTOUTP09_PUF.txt')
#VTOUTP10.TXT missing header, so header was added
out10 = read_patient('VTOUTP10_revised.TXT')
out11 = read_patient('VTOUTP11.TXT')
out12 = read_patient('VTOUTP12.TXT')
out13 = read_patient('VTOUTP13.TXT')
out14 = read_patient('VTOUTP14.TXT')
out15 = read_patient('VTOUTP15_revised.TXT')
out16 = read_patient('VTOUTP16.TXT')

outpatient = pd.concat([out09,out10,out11,out12,out13,out14,out15,out16], ignore_index=True)
outpatient.to_csv('2009-2016outpatient.csv')

/Users/rosaria/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (8,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/rosaria/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (8,9,29,54,55,56,57,58,59,71) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/rosaria/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (2,8,9,29,54,55,56,57,58,59,71) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/rosaria/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (2,5,8,54,55,71) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/rosaria/anaconda

In [9]:
outpatient['type'] = 'out'
inpatient['type'] = 'in'
extended['type'] = 'ex'
combined = pd.concat([outpatient, inpatient, extended], ignore_index=True)
combined.head()

,SEX,CCSDX,DY,DISCD_QTR,type
0,2,184,2009.0,4,out
1,2,177,2009.0,4,out
2,2,176,2009.0,4,out
3,2,176,2009.0,4,out
4,2,176,2009.0,4,out


## III. Analysis

Analyze longitudinally

In [10]:
by_qtr_type = combined.groupby(['DY', 'DISCD_QTR', 'type', 'CCSDX']).size().reset_index(name='counts')
by_qtr_type.to_csv('2009-2016_diagnosis_by_qtr_type.csv')

To normalize, Mukul got the number of women age between 15-45 from https://factfinder.census.gov/faces/nav/jsf/pages/index.xhtml American Communities Survey. 

In [11]:
women = pd.read_csv('reproductiveWomen.csv')
women = women.set_index('Year')

In [12]:
normFactor = women['Females in our range'] / women['Females in our range'][2016]
normDict = normFactor.to_dict()

normalized = by_qtr_type.copy()

normFactors = normalized['DY'].apply(lambda x: normDict[x])
normalized = normalized.assign(normFactor = normFactors)
normalized.head()
normalized['norm_counts'] = normalized['counts'] * normalized['normFactor']
normalized.head()

,DY,DISCD_QTR,type,CCSDX,counts,normFactor,norm_counts
0,2009.0,1,ex,176,883,1.061029,936.888556
1,2009.0,1,ex,177,125,1.061029,132.628618
2,2009.0,1,ex,178,13,1.061029,13.793376
3,2009.0,1,ex,179,1,1.061029,1.061029
4,2009.0,1,ex,180,71,1.061029,75.333055


In [13]:
# add date column for Tableau analysis 
# qtr 1: Jan 1, qtr2: Apr 1, qtr3: July 1, qtr4: Oct 1

normalized['DY'] = normalized['DY'].astype('int64')

def qtr_to_month(qtr):
    if qtr == 1: return 1
    elif qtr == 2: return 4
    elif qtr == 3: return 7
    else: return 10

month = normalized['DISCD_QTR'].map(qtr_to_month)
normalized = normalized.assign(month = month)
normalized['day'] = 1
normalized.rename(columns={'DY': 'year'}, inplace=True)

normalized = normalized.assign(date = pd.to_datetime(normalized[['year', 'month', 'day']]))
normalized.to_csv('2009-2016_diagnosis_by_qtr_type_norm_date.csv')
normalized.head()

,year,DISCD_QTR,type,CCSDX,counts,normFactor,norm_counts,month,day,date
0,2009,1,ex,176,883,1.061029,936.888556,1,1,2009-01-01
1,2009,1,ex,177,125,1.061029,132.628618,1,1,2009-01-01
2,2009,1,ex,178,13,1.061029,13.793376,1,1,2009-01-01
3,2009,1,ex,179,1,1.061029,1.061029,1,1,2009-01-01
4,2009,1,ex,180,71,1.061029,75.333055,1,1,2009-01-01


In [14]:
def find_outcome(dx):
    if dx == 176: return 'pre'
    elif dx == 196: return 'pos'
    else: return 'neg'

normalized = normalized.assign(outcome = normalized['CCSDX'].map(find_outcome))
normalized.head()

,year,DISCD_QTR,type,CCSDX,counts,normFactor,norm_counts,month,day,date,outcome
0,2009,1,ex,176,883,1.061029,936.888556,1,1,2009-01-01,pre
1,2009,1,ex,177,125,1.061029,132.628618,1,1,2009-01-01,neg
2,2009,1,ex,178,13,1.061029,13.793376,1,1,2009-01-01,neg
3,2009,1,ex,179,1,1.061029,1.061029,1,1,2009-01-01,neg
4,2009,1,ex,180,71,1.061029,75.333055,1,1,2009-01-01,neg


In [15]:
# for slide 2, get stats on total number from 2009 to 2016
total_count = normalized[['date', 'outcome', 'norm_counts']]
total_count = total_count.pivot_table(index = 'date', columns = 'outcome', aggfunc=np.sum)
total_count

norm_counts                          
outcome             neg          pos          pre
date                                             
2009-01-01  2146.461550  7501.474620  1042.991450
2009-04-01  2300.310746  6795.890374   673.753378
2009-07-01  2288.639428  6822.416097   669.509262
2009-10-01  2160.254926  6959.288831   597.359294
2011-01-01  1903.425800  8457.500490   711.215256
2011-04-01  2033.952299  6855.210629   625.910536
2011-07-01  2234.367003  6759.628232   640.299284
2011-10-01  2030.868996  6331.049097   628.993839
2012-01-01  1958.919359  6560.599016   714.369553
2012-04-01  2154.302484  6593.162870   711.316692
2012-07-01  2087.139535  6282.788634   735.739583
2012-10-01  2087.139535  6361.145408   749.986269
2013-01-01  2051.035413  6908.857220   742.616270
2013-04-01  2193.496494  6744.168169   832.538513
2013-07-01  2354.144095  7154.375252   803.238007
2013-10-01  2070.232296  7050.307938   871.942641
2014-01-01  2133.692534  7165.986247   792.083030
2014-04-01  2124.634405  6927.455525   851.464096
2014-07-01  2467.836837  7448.801153  1039.671881
2014-10-01  2230.312573  7147.869989  1199.698821
2015-01-01  2280.740438  6757.861034  1244.314136
2015-04-01  2094.946964  6541.938888  1356.794510
2015-07-01  2192.363002  6324.008164  1268.417073
2015-10-01  2122.062768  6826.152690  1189.078238
2016-01-01  2261.000000  7483.000000  1379.000000
2016-04-01  2088.000000  7211.000000  1531.000000
2016-07-01  2198.000000  5977.000000  1024.000000
2016-10-01  2090.000000  5822.000000  1022.000000

In [16]:
total_count.mean()

             outcome
norm_counts  neg        2154.938553
             pos        6848.962020
             pre         916.046486
dtype: float64

In [17]:
total_count.std()

             outcome
norm_counts  neg        120.819645
             pos        525.044200
             pre        268.570562
dtype: float64